# ELP EU Orders

#### Imports

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from nv_forecasting.configs import DEFAULT_CONFIG as config
from nv_forecasting.load_data.elp_eu_orders_daily import ELPEUOrdersDaily
from nv_forecasting.style import set_style
from nv_forecasting.statistics.white_noise_test import white_noise_test

#### Settings

In [ ]:
TARGET = config.get('target') # Target column name
AGGREGATION = config.get('aggregation')

# Style
set_style()

#### Loading the data

In [ ]:
data_handler = ELPEUOrdersDaily('data\\elp_eu_orders_daily.csv')
df = data_handler.get_dataframe(agg=AGGREGATION)
weekly_df = df.resample('W').sum()
monthly_df = df.resample('ME').sum()

#### Initial data exploring

##### Relationships with time

In [ ]:
for word in ['day', 'day_of_week', 'month', 'year']:
    fig, ax = plt.subplots(1, 1, figsize=(12, 3))
    sns.lineplot(df, x=getattr(df.index, word), y=df[TARGET])
    plt.xlabel('Date')
    plt.ylabel('$', rotation=0, labelpad=16)
    ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    plt.title(f'Mean value of sum of orders for each day, aggregated by {word}')
    plt.show()

##### ACF and PACF

In [ ]:
fig, (acf, pacf) = plt.subplots(2, 1, figsize=(20, 6))

plot_acf(df[TARGET], ax=acf, lags=35)
plot_pacf(df[TARGET], ax=pacf, lags=35)
plt.suptitle('Orders in $ – daily')
plt.show()

In [ ]:
fig, (acf, pacf) = plt.subplots(2, 1, figsize=(20, 6))

plot_acf(weekly_df[TARGET], ax=acf, lags=52)
plot_pacf(weekly_df[TARGET], ax=pacf, lags=52)
plt.suptitle('Orders in $ – weekly')
plt.show()

In [ ]:
fig, (acf, pacf) = plt.subplots(2, 1, figsize=(20, 6))

plot_acf(monthly_df[TARGET], ax=acf, lags=24)
plot_pacf(monthly_df[TARGET], ax=pacf, lags=24)
plt.suptitle('Orders in $ – monthly')
plt.show()

##### Ljung–Box test – checking if data is similar to white noise 

In [ ]:
print('--- Daily aggregation ---')
white_noise_test(df[TARGET], lags=35)

print('--- Weekly aggregation ---')
white_noise_test(weekly_df[TARGET], lags=52)

print('--- Monthly aggregation ---')
white_noise_test(monthly_df[TARGET], lags=24)

Found weekly seasonality and also yearly seasonality, but only on the weekly level.